### Import Modules, Assign Data Path & Read into GDF

In [17]:
#Modules
import osmnx as ox 
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import matplotlib.pyplot as plt
from descartes import PolygonPatch
ox.config(log_console=True, use_cache=True)
ox.__version__
#Read Data into DataFrame
vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')
# Convert DataFrame to GeoDataFrame
vax_gdf = gpd.GeoDataFrame(vax_df, geometry=gpd.points_from_xy(vax_df['X'], vax_df['Y']))
#Reproject to UTM 17
vax_gdf.crs = 'EPSG:32617'

2022-02-27 16:27:28 Configured OSMnx 1.1.2
2022-02-27 16:27:28 HTTP response caching is on


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/4099419668.py:16: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')


### Limit to Detroit Metro Area (Which we are defining as Wayne, Oakland, and Macomb County)

In [18]:
#limit by state to Michigan
vax_MI = vax_gdf[vax_gdf['State']== 'MI'] 
#Limit By County to Wayne, Oakland, and Macomb
vax_wayne = vax_MI[vax_MI['county']== 'Wayne']
vax_oakland = vax_MI[vax_MI['county']== 'Oakland'] 
vax_macomb = vax_MI[vax_MI['county']== 'Macomb']
#Agglomerate to one GDF of Detroit Metro
vax_detr_metro = vax_wayne 
vax_detr_metro = vax_detr_metro.append(vax_oakland)
vax_detr_metro = vax_detr_metro.append(vax_macomb)

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/2395504573.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_oakland)
/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/2395504573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_macomb)


In [19]:
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index('id') #could be here that somethign goes wrong

    edge_lines = []
    for n_fr, n_to in subgraph.edges():
        f = nodes_gdf.loc[n_fr].geometry
        t = nodes_gdf.loc[n_to].geometry
        edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
        edge_lines.append(edge_lookup)

    n = nodes_gdf.buffer(node_buff).geometry
    e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
    all_gs = list(n) + list(e)
    new_iso = gpd.GeoSeries(all_gs).unary_union
        
    # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
    if infill:
        new_iso = Polygon(new_iso.exterior)
    isochrone_polys.append(new_iso)
    return isochrone_polys

In [20]:
def isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute):
    network_type = network_type
    trip_time = trip_time
    travel_speed = travel_speed
    meters_per_minute = meters_per_minute
    # download the street network
    G = ox.graph_from_point(place, network_type=network_type)
    # find the centermost node and then project the graph to UTM
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    x, y = gdf_nodes['geometry'].unary_union.centroid.xy
    center_node = ox.get_nearest_node(G, (y[0], x[0]))
    G = ox.project_graph(G)
    for u, v, k, data in G.edges(data=True, keys=True):
        data['time'] = data['length'] / meters_per_minute
    isochrone_polys = []
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)
    isochrone_polys = make_iso_polys(G, center_node, edge_buff=25, node_buff=0, infill=True)
    return isochrone_polys

In [25]:
#Set Variables
network_type = 'drive'
trip_time = 10 #in minutes 
travel_speed = 48.3 #walking speed in km/hour '''was 4.5 for walking'''
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
detroit_polygons = gpd.GeoDataFrame(crs='EPSG:32617')# add geometry column? 

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/46855611.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  detroit_polygons = gpd.GeoDataFrame(crs='EPSG:32617')# add geometry column?


In [26]:
for i in range(len(vax_detr_metro)):
    place = vax_detr_metro['Y'].iloc[i],vax_detr_metro['X'].iloc[i]
    isochrone_polys=isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute)
    gdf_iso_poly = gpd.GeoDataFrame(index=[i], crs='EPSG:32617', geometry=isochrone_polys)
    detroit_polygons = detroit_polygons.append(gdf_iso_poly)
detroit_polygons.to_file('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Shapefile_Outputs/10_min_drive_polys.shp')

2022-02-27 17:27:19 Created bbox 1000 m from (42.3963769002206, -83.4806078996323): 42.40537010357553,42.387383696865676,-83.46843020434454,-83.49278559492006
2022-02-27 17:27:19 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:27:19 Projected GeoDataFrame to epsg:4326
2022-02-27 17:27:19 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:27:19 Projected GeoDataFrame to epsg:4326
2022-02-27 17:27:19 Requesting data within polygon from API in 1 request(s)
2022-02-27 17:27:19 Retrieved response from cache file "cache/aa19c8cd2d450ef7e61de2faf88041e24b3a921e.json"
2022-02-27 17:27:19 Got all network data within polygon from API in 1 request(s)
2022-02-27 17:27:19 Creating graph from downloaded OSM data...
2022-02-27 17:27:19 Created graph with 2227 nodes and 4009 edges
2022-02-27 17:27:19 Added length attributes to graph edges
2022-02-27 17:27:19 Identifying all no

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/4006269401.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)


2022-02-27 17:27:21 Created bbox 1000 m from (42.3903256997602, -82.9147948003165): 42.39931890311512,42.38133249640527,-82.90261827909424,-82.92697132153876
2022-02-27 17:27:21 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:27:21 Projected GeoDataFrame to epsg:4326
2022-02-27 17:27:21 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:27:21 Projected GeoDataFrame to epsg:4326
2022-02-27 17:27:21 Requesting data within polygon from API in 1 request(s)
2022-02-27 17:27:21 Retrieved response from cache file "cache/68c53ca9e02df6ae619b21c1a2726e1fcb11d788.json"
2022-02-27 17:27:21 Got all network data within polygon from API in 1 request(s)
2022-02-27 17:27:21 Creating graph from downloaded OSM data...
2022-02-27 17:27:21 Created graph with 1728 nodes and 3696 edges
2022-02-27 17:27:21 Added length attributes to graph edges
2022-02-27 17:27:21 Identifying all nod

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-27 17:27:22 Created edges GeoDataFrame from graph
2022-02-27 17:27:22 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:27:22 Created graph from node/edge GeoDataFrames
2022-02-27 17:27:22 Projected graph with 183 nodes and 539 edges


...

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/4006269401.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)


2022-02-27 17:37:23 Created bbox 1000 m from (42.7144840001097, -83.0367249996309): 42.723477203464626,42.70549079675477,-83.02448507128297,-83.04896492797884
2022-02-27 17:37:23 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:37:23 Projected GeoDataFrame to epsg:4326
2022-02-27 17:37:23 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:37:23 Projected GeoDataFrame to epsg:4326
2022-02-27 17:37:23 Requesting data within polygon from API in 1 request(s)
2022-02-27 17:37:23 Retrieved response from cache file "cache/4f23f76a182c2a4092d58e492ce25488da310745.json"
2022-02-27 17:37:23 Got all network data within polygon from API in 1 request(s)
2022-02-27 17:37:23 Creating graph from downloaded OSM data...
2022-02-27 17:37:23 Created graph with 2153 nodes and 4040 edges
2022-02-27 17:37:23 Added length attributes to graph edges
2022-02-27 17:37:23 Identifying all no

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-27 17:37:24 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-27 17:37:24 Created graph from node/edge GeoDataFrames
2022-02-27 17:37:24 Projected graph with 169 nodes and 357 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_5374/4006269401.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)
/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


""
